In [ ]:
!pip install python-binance
!pip install binance-client

     |████████████████████████████████| 63 kB 667 kB/s 
     |████████████████████████████████| 1.1 MB 10.7 MB/s 
     |████████████████████████████████| 103 kB 44.0 MB/s 
     |████████████████████████████████| 216 kB 57.3 MB/s 
     |████████████████████████████████| 288 kB 57.4 MB/s 
     |████████████████████████████████| 271 kB 56.5 MB/s 
     |████████████████████████████████| 160 kB 61.2 MB/s 
     |████████████████████████████████| 192 kB 56.8 MB/s 


# Simple Binance Data Fetch

In [ ]:
import binance.client
import pandas as pd
import numpy as np
import time
import os
import requests
from pprint import pprint
import subprocess
import datetime as dt
import json
import pandas as pd

from xgboost import XGBClassifier
from xgboost import XGBRegressor
from tqdm import tqdm

class BinanceClient():

    INTERVAL_1MINUTE = binance.client.Client.KLINE_INTERVAL_1MINUTE

    column_names = ['open_time',
            'open',
            'high',
            'low',
            'close',
            'vol',
            'close_time',
            'quote_asset_volume',
            'number_of_trades',
            'taker_buy_base_asset_volume',
            'taker_buy_quote_asset_volume',
            'ignore']

    def __init__(self, api_key, api_secret):
        self.client = binance.client.Client(api_key, api_secret)

    def polish_kline(self, raw_kline):
        kline_d =  {'open_time':    raw_kline[0],
                    'open':         raw_kline[1],
                    'close':        raw_kline[2],
                    'low':          raw_kline[3],
                    'high':         raw_kline[4],
                    'vol':          raw_kline[5]}
        kline_d['open_datetime'] = pd.to_datetime(kline_d['open_time'], unit='ms')
        return kline_d

    def historical_klines_generator(self, instrument, interval, start_dt, end_dt=None):
        for kline in self.client.get_historical_klines_generator(instrument, interval, str(start_dt), str(end_dt) if end_dt else None):
            yield self.polish_kline(kline)


In [ ]:
api_key = "vT0XoiF4FbrpHbH04ZEew6A9jOORd79J4RdmId0CTT6wwX97EuY9hPCBcp9F904f"
api_secret = "wcpF4Ne1tNGbq5vPx2oaXrpbE2JnNXhn6tKV6I32uKNdBMuEazRfDPiMJkIVqVr8"
        

In [ ]:
client = BinanceClient(api_key, api_secret)

start_date = dt.datetime(2021,11,1,10,0,0)
instrument = "BTCUSDT"

kline_df = pd.DataFrame(columns=["open","high","low","close","vol"])
for kline in client.historical_klines_generator(instrument, BinanceClient.INTERVAL_1MINUTE, start_date): 
    kline_df.loc[kline['open_datetime']] = (kline["open"], kline["high"], kline["low"], kline["close"], kline["vol"])

In [ ]:
kline_df

,open,high,low,close,vol
2021-11-01 10:00:00,61950.45000000,62028.71000000,61929.40000000,62030.00000000,33.71812000
2021-11-01 10:01:00,62028.71000000,62048.83000000,62001.34000000,62049.54000000,21.69355000
2021-11-01 10:02:00,62048.83000000,62026.57000000,62018.40000000,62061.35000000,13.21239000
2021-11-01 10:03:00,62026.57000000,62003.79000000,61964.22000000,62028.72000000,66.53268000
2021-11-01 10:04:00,62003.79000000,62005.70000000,61959.93000000,62027.86000000,36.27341000
...,...,...,...,...,...
2021-12-06 21:36:00,49195.75000000,49226.67000000,49193.11000000,49237.34000000,22.65868000
2021-12-06 21:37:00,49226.67000000,49244.73000000,49217.75000000,49256.61000000,32.09326000
2021-12-06 21:38:00,49244.73000000,49246.48000000,49182.92000000,49261.54000000,36.86333000
2021-12-06 21:39:00,49246.48000000,49293.27000000,49220.86000000,49298.00000000,30.23855000


# Simple ML

In [ ]:
ml_df = pd.DataFrame()
for i in range(60):
  ml_df[f"t_{i}"] = kline_df.close.shift(i)

In [ ]:
ml_df = ml_df.dropna().astype(float)

One row of data represents the price movements in one hour.

In [ ]:
ml_df.t_0

2021-11-01 10:59:00    62273.27
2021-11-01 11:00:00    62298.99
2021-11-01 11:01:00    62276.55
2021-11-01 11:02:00    62300.00
2021-11-01 11:03:00    62400.00
                         ...   
2021-12-06 21:36:00    49237.34
2021-12-06 21:37:00    49256.61
2021-12-06 21:38:00    49261.54
2021-12-06 21:39:00    49298.00
2021-12-06 21:40:00    49328.96
Name: t_0, Length: 51042, dtype: float64

# Bollinger Bands

In [ ]:
class BollingerBands:
    def __init__(self, df, k_sigma=3):
        self.k_sigma = k_sigma
        self.boll_df = df
        self.boll_df["ma"] = df.price.rolling(300, min_periods=1).mean()
        self.boll_df["std"] = df.price.rolling(300, min_periods=1).std()
        self.boll_df["bb_u"] = df["ma"] + self.k_sigma*df["std"]
        self.boll_df["bb_l"] = df["ma"] - self.k_sigma*df["std"]

    def updateInds(self, new_signal):
        '''
        Requires optimization - compute rolling only for the last entry
        '''
        price_series = self.boll_df.iloc[-300+1:].price
        price_series = price_series.append(new_signal.price)
        ma = price_series.rolling(300).mean()[-1]
        std = price_series.rolling(300).std()[-1]
        bb_u = ma + self.k_sigma*std
        bb_l = ma - self.k_sigma*std
        # hitting lower band from above
        cross_signal = (self.boll_df.price[-1] > self.boll_df.bb_l[-1]) & (price_series[-1] < bb_l)
        # hitting rolling average from below
        ma_cross_signal = (self.boll_df.price[-1] < self.boll_df.ma[-1]) & (price_series[-1] > ma)
        signal = pd.DataFrame({"price": new_signal.price, "ma": ma, "std": std, "bb_u": bb_u, "bb_l": bb_l, "cross_signal": cross_signal, "ma_cross_signal": ma_cross_signal}, index=[new_signal.index[-1]])
        self.boll_df = self.boll_df.append(signal)

    def getSignal(self):
        if self.boll_df.iloc[-1].cross_signal:
            return 1
        elif self.boll_df.iloc[-1].ma_cross_signal:
            return -1
        else:
            return 0

In [ ]:
class InvestmentManager:
    def __init__(self, initial_history, usd=1000, btc=0, loss_treshold=0.05, fee_ratio=1e-3, k_sigma=3):
        self.wallet = pd.DataFrame({'usd': [1000.], 'btc': [0.], 'usd_total_equivalent': 1000.}, index=[initial_history.index[-1]])
        self.loss_treshold = loss_treshold
        self.bb = BollingerBands(initial_history, k_sigma=k_sigma)
        self.history = initial_history
        self.last_investment_price = None
        self.fee_ratio = fee_ratio

    def evaluateWallet(self):
        return self.wallet.iloc[-1].usd_total_equivalent

    def timeStep(self, new_signal):
        self.history = self.history.append(new_signal)
        # We could add iterarion over all methods
        self.bb.updateInds(new_signal)
        signal = self.bb.getSignal()
        # Responding to signal from the Bollinger Bands and buying BTC all-in
        if signal == 1 and self.wallet.iloc[-1].usd>0:
            btc = (1-self.fee_ratio) * self.wallet.iloc[-1].usd/self.history.iloc[-1].price
            self.wallet = self.wallet.append(pd.DataFrame({'usd': 0., 'btc': btc, 'usd_total_equivalent': btc * self.history.iloc[-1].price}, index=[self.history.index[-1]]))
            self.last_investment_price = self.history.iloc[-1].price

        elif self.last_investment_price is not None and (signal == -1 or (signal == 0 and new_signal.price.iloc[-1] <= (1-self.loss_treshold)*self.last_investment_price)):
            usd = (1-self.fee_ratio) * self.wallet.iloc[-1].btc * self.history.iloc[-1].price
            self.wallet = self.wallet.append(pd.DataFrame({'usd': usd, 'btc': 0, 'usd_total_equivalent': usd}, index=[self.history.index[-1]]))
            self.last_investment_price = None
        else:
            self.wallet = self.wallet.append(pd.DataFrame({'usd': self.wallet.iloc[-1].usd, 'btc': self.wallet.iloc[-1].btc, 'usd_total_equivalent': self.wallet.iloc[-1].usd + self.wallet.iloc[-1].btc*self.history.iloc[-1].price}, index=[self.history.index[-1]]))


## Hoarder

In [ ]:
class Hoarder:
    def __init__(self, initial_history, usd=1000, btc=0,fee_ratio=1e-3):
        self.history = initial_history
        self.wallet = pd.DataFrame({'usd': [0.], 'btc': [usd/self.history.iloc[-1].price *(1-fee_ratio)], 'usd_total_equivalent': 1000*(1-fee_ratio)}, index=[initial_history.index[-1]])
        self.fee_ratio = fee_ratio

    def evaluateWallet(self):
        return self.wallet.iloc[-1].usd_total_equivalent

    def timeStep(self, new_signal):
        self.history = self.history.append(new_signal)
        btc = self.wallet.iloc[-1].btc
        self.wallet = self.wallet.append(pd.DataFrame({'usd': 0., 'btc': btc, 'usd_total_equivalent': btc * self.history.iloc[-1].price}, index=[self.history.index[-1]]))

## Backtesting

In [ ]:
boll_df = pd.DataFrame(index=kline_df.index)
boll_df["price"] = kline_df.close.astype(float)

In [ ]:
initial_hist = boll_df.iloc[:300]

## Kalman Filter

In [ ]:
# class KalmanFilter:
#     def __init__(self, model_matrix, latent_to_obs, process_noise_cov, measurement_noise_cov, initial_latent):
#         self.model_matrix = model_matrix.astype('float64')
#         self.latent_to_obs = latent_to_obs.astype('float64')
#         self.proc_noise_cov = process_noise_cov.astype('float64')
#         self.mea_noise_cov = measurement_noise_cov.astype('float64')
#         self.last_prediction = initial_latent.astype('float64')
#         self.prediction_err_cov = np.array([[1, 0], [0, 1]]).astype('float64')
#         self.predictions = []
    
#     def prediction_phase(self):
#         # noise = np.random.multivariate_normal(mean=np.array([0, 0]), cov=self.proc_noise_cov,)
#         self.pred_precor = self.model_matrix @ self.last_prediction #+ noise
#         self.prediction_err_cov = self.model_matrix @ self.prediction_err_cov @ self.model_matrix.T + self.proc_noise_cov
        
#     def correction_phase(self, new_observation):
#         pre_fit_residua = new_observation - self.latent_to_obs @ self.pred_precor
#         innovation_cov = self.mea_noise_cov + self.latent_to_obs @ self.prediction_err_cov @ self.latent_to_obs.T
#         kalman_gain = self.prediction_err_cov @ self.latent_to_obs
        
#         if innovation_cov.shape[0] > 1:
#             kalman_gain = kalman_gain @ np.linalg.inv(innovation_cov)
#             kalman_gain = kalman_gain.reshape(self.pred_precor.shape)
#             corr_pred = self.pred_precor + kalman_gain @ pre_fit_residua
#             kalman_gain_diff = (np.eye(N=self.prediction_err_cov.shape[0]) - kalman_gain @ self.latent_to_obs)
#             self.prediction_err_cov = (kalman_gain_diff @ self.prediction_err_cov @ kalman_gain_diff.T
#                                        + kalman_gain @ self.mea_noise_cov @ kalman_gain.T)
#         else: 
#             kalman_gain = kalman_gain * np.linalg.inv(innovation_cov)
#             kalman_gain = kalman_gain.reshape(self.pred_precor.shape)
#             corr_pred = self.pred_precor + kalman_gain * pre_fit_residua
#             kalman_gain_diff = (np.eye(N=self.prediction_err_cov.shape[0]) - kalman_gain @ self.latent_to_obs)
#             self.prediction_err_cov = (kalman_gain_diff @ self.prediction_err_cov @ kalman_gain_diff.T
#                                        + kalman_gain * self.mea_noise_cov * kalman_gain.T)

#         self.prediction_err_cov = (np.eye(N=self.prediction_err_cov.shape[0]) - kalman_gain @ self.latent_to_obs) @ self.prediction_err_cov
#         y = self.latent_to_obs @ np.ravel(corr_pred) + np.random.multivariate_normal(mean=np.array([0.]), cov=self.mea_noise_cov)

#         return corr_pred, y

#     def timeStep(self, new_observation):
#         self.prediction_phase()
#         pred, pred_obs = self.correction_phase(new_observation)
#         self.last_prediction = pred
#         self.predictions.append(pred)

In [ ]:
class KalmanFilter:
    def __init__(self, initial_state, state_uncertainty, sigma_acceleration, process_noise=0.0):
        self.state_transition_matrix = np.array([[1.0, 1.0, 0.5],[0.0, 1.0, 1.0], [0.0, 0.0, 1.0]])
        self.input_transition_matrix = None
        # self.process_noise = process_noise
        self.process_noise_matrix = np.array([[0.25, 0.5, 0.5], [0.5, 1., 1.], [0.5, 1., 1.]]) * sigma_acceleration
        self.last_state = initial_state
        self.last_predicted_state = None
        self.last_state_uncertainty = state_uncertainty
        self.last_prediction_uncertainty = None
        # self.last_predicted_state = np.array()
        self.predicted_states = []
        self.prediction_uncertainties = []
        self.observation_matrix = np.array([[1.0, 0.0, 0.0]])
        # self.observation_matrix = 1.0
        self.measurements = []
        self.state_history = []
        self.kalman_gain_history = []

    def predict(self, measurement, measurement_noise):
        self.measurements.append(measurement)
        # state extrapolation equation
        # self.last_predicted_state = self.state_transition_matrix @ self.last_state + \
        #             self.input_transition_matrix @ measurement + self.process_noise
        self.last_predicted_state = self.state_transition_matrix @ self.last_state
        self.predicted_states.append(self.last_predicted_state)
        # covariance extrapolation equation
        self.last_prediction_uncertainty = self.state_transition_matrix @ self.last_state_uncertainty @ \
                    self.state_transition_matrix.T + self.process_noise_matrix
        self.prediction_uncertainties.append(self.last_prediction_uncertainty)
        # measurement equation


    def correct(self, measurement, measurement_noise):
        # kalman gain
        temp = self.observation_matrix @ self.last_prediction_uncertainty @ self.observation_matrix.T + measurement_noise
        if type(temp) is np.ndarray:
            self.kalman_gain = self.last_prediction_uncertainty @ self.observation_matrix.T @ np.linalg.inv(temp)
        else:
            self.kalman_gain = self.last_prediction_uncertainty @ self.observation_matrix.T * (1/temp)
        self.kalman_gain_history.append(self.kalman_gain)
        
        # state update
        self.last_state = self.last_predicted_state + self.kalman_gain @ (measurement - self.observation_matrix @ self.last_predicted_state)
        self.state_history.append(self.last_state[0,:])

        # state uncertainty
        temp = self.kalman_gain @ self.observation_matrix
        I = np.diag(np.ones(temp.shape[0]))
        if type(measurement_noise) is np.ndarray:
            self.last_state_uncertainty = (I - temp) @ self.last_prediction_uncertainty @ (I - temp).T + \
                self.kalman_gain @ measurement_noise @ self.kalman_gain.T
        else:
            self.last_state_uncertainty = (I - temp) @ self.last_prediction_uncertainty @ (I - temp).T + \
                self.kalman_gain @ self.kalman_gain.T * measurement_noise


    def timeStep(self, measurement, measurement_noise):
        self.predict(measurement, measurement_noise)
        self.correct(measurement, measurement_noise)

    def plotKalmanGain(self):
        plt.figure(figsize=(16, 9))
        plt.plot(np.array(self.kalman_gain_history)[:, 0, 0])
        plt.show()

    def plotExtrapolationVsMeasurement(self):
        plt.figure(figsize=(16, 9))
        plt.plot(self.measurements, )
        plt.plot(np.array(self.state_history)[:, 0])
        # print(len(self.measurements), len(self.predictions_history))
        plt.legend(["Measurements", "Estimations"])
        plt.show()

In [ ]:
sigma_acc = 100.0
measurement_noise = 200
process_noise = 0
x0 = np.array([[62000, 0, 0]]).T
P0 = np.array([[500**2, 0.0, 0.0], [0.0, 500**2, 0.0], [0.0, 0.0, 500**2]])
kalman = KalmanFilter(x0, P0, sigma_acc, process_noise)
for id, entry in boll_df.iterrows():
    sample = pd.DataFrame([entry], index=[id])
    kalman.timeStep(sample.price.to_numpy()[0], measurement_noise**2)

In [ ]:
class KalmanManager:
    def __init__(self, kalman_filter, initial_time, usd=1000, btc=0, fee_ratio=1e-3, treshold=100., price_normalization_factor=1e3):
        self.wallet = pd.DataFrame({'usd': [1000.], 'btc': [0.], 'usd_total_equivalent': 1000.}, index=[initial_time])
        self.in_position = False
        self.fee_ratio = fee_ratio
        self.kalman = kalman_filter
        self.treshold = treshold
        self.price_normalization_factor = price_normalization_factor
        self.history = pd.DataFrame({'price': []})

    def evaluateWallet(self):
        return self.wallet.iloc[-1].usd_total_equivalent

    def timeStep(self, new_signal):
        new_signal = new_signal/self.price_normalization_factor
        self.history = self.history.append(new_signal)
        new_price = new_signal.price.to_numpy()/self.price_normalization_factor
        # We could add iterarion over all methods
        self.kalman.timeStep(new_price)
        prediction = np.array(self.kalman.predictions)[-1, 0]*self.price_normalization_factor
        # Responding to signal from the Bollinger Bands and buying BTC all-in
        # print(prediction, new_price, self.treshold, self.price_normalization_factor)
        if not self.in_position and prediction>new_price+self.treshold/self.price_normalization_factor:
            btc = (1-self.fee_ratio) * self.wallet.iloc[-1].usd/(new_price*self.price_normalization_factor)
            self.wallet = self.wallet.append(pd.DataFrame({'usd': 0., 'btc': btc, 'usd_total_equivalent': btc * (new_price*self.price_normalization_factor)}, index=[self.history.index[-1]]))
            self.last_investment_price = (new_price*self.price_normalization_factor)
            self.in_position = True

        elif self.in_position and prediction<new_price+self.treshold/self.price_normalization_factor:
            usd = (1-self.fee_ratio) * self.wallet.iloc[-1].btc * (new_price*self.price_normalization_factor)
            self.wallet = self.wallet.append(pd.DataFrame({'usd': usd, 'btc': 0, 'usd_total_equivalent': usd}, index=[self.history.index[-1]]))
            self.last_investment_price = None
            self.in_position = False
        else:
            self.wallet = self.wallet.append(pd.DataFrame({'usd': self.wallet.iloc[-1].usd, 'btc': self.wallet.iloc[-1].btc, 'usd_total_equivalent': self.wallet.iloc[-1].usd + self.wallet.iloc[-1].btc*(new_price*self.price_normalization_factor)}, index=[self.history.index[-1]]))


In [ ]:
kalman_manager = KalmanManager(kalman, initial_time=boll_df.index[0], treshold=50.)

In [ ]:
boll_df.iloc[:5]

,price
2021-11-01 10:00:00,62030.00
2021-11-01 10:01:00,62049.54
2021-11-01 10:02:00,62061.35
2021-11-01 10:03:00,62028.72
2021-11-01 10:04:00,62027.86


In [ ]:
import xgboost as xgb
import sklearn.metrics
import dateutil.parser

In [ ]:
class DataSeries:
    def __init__(self, data):
        self.data = pd.DataFrame(data)
        self.data_supervised = None
        self.data_mean = dict()
        self.data_std = dict()
        self.data_is_normalized = dict()
        for column in self.data.columns:
            self.data_mean[column] = self.data[column].mean()
            self.data_std[column] = self.data[column].std()
            self.data_is_normalized[column] = False

    def add_column_of_data(self, new_data, column="prize"):
        self.data[column] = new_data
        self.data_mean[column] = self.data[column].mean()
        self.data_std[column] = self.data[column].std()
        self.data_is_normalized[column] = False

    def normalize_column(self, column):
        self.data[column] = (self.data[column] - self.data_mean[column]) / self.data_std[column]
        self.data_is_normalized[column] = True

    def denormalize_column(self, column):
        self.data[column] = self.data[column] * self.data_std[column] + self.data_mean[column]
        self.data_is_normalized[column] = False

    def normalize_columns(self):
        for column in self.data.columns:
            if not self.data_is_normalized[column]:
                self.data[column] = (self.data[column] - self.data_mean[column]) / self.data_std[column]
                self.data_is_normalized[column] = True

    def denormalize_columns(self):
        for column in self.data.columns:
            if self.data_is_normalized[column]:
                self.data[column] = self.data[column] * self.data_std[column] + self.data_mean[column]
                self.data_is_normalized[column] = False

    def series_to_supervised(self, n_in=50, n_out=1, dropnan=True, target_columns=["price"]):
        n_vars = 1 if type(self.data) is list else self.data.shape[1]
        cols = list()
        for i in range(n_in, 0, -1):
            cols.append(self.data.shift(i))
        for i in range(0, n_out):
            cols.append(self.data[target_columns].shift(-i))
        agg = pd.concat(cols, axis=1)
        if dropnan:
            agg.dropna(inplace=True)
        self.__rename_supervised_column_names(agg, n_vars, n_in, n_out, target_columns)
        self.target_columns = target_columns
        self.n_in = n_in
        self.n_out = n_out
        self.data_supervised = agg

    def __rename_supervised_column_names(self, data_supervised, n_vars, n_in, n_out, target_columns):
        new_column_names = []
        for i in range(n_in):
            for column in self.data.columns:
                new_column_names.append(f'{column}.{-(n_in-1-i)}')
        for i in range(n_out):
            for column in target_columns:
                new_column_names.append(f'target.{column}.{i}')
        data_supervised.columns = new_column_names

    def normalize_window(self):
        for basic_column_name in self.data.columns:
            divider_column_name = basic_column_name + ".0"
            for column in self.data_supervised.columns:
                if "target" != column[:6]:
                    if basic_column_name == column[:len(basic_column_name)] and divider_column_name != column:
                        self.data_supervised[column] /= self.data_supervised[divider_column_name]
                else:
                    if basic_column_name == column[7:7+len(basic_column_name)]:
                        self.data_supervised[column] /= self.data_supervised[divider_column_name]
            self.data_supervised[divider_column_name] = 1.0

    def regression_to_classification(self):
        for basic_column_name in self.data.columns:
            last_observed_value_column_name = basic_column_name + ".0"
            for column in self.data_supervised.columns:
                if "target" == column[:6] and basic_column_name == column[7:7+len(basic_column_name)]:
                        self.data_supervised[column] = (self.data_supervised[last_observed_value_column_name] < self.data_supervised[column]).astype(int)

    def normalize_window_testset(self, testset):
        for basic_column_name in self.data.columns:
            divider_column_name = basic_column_name + ".0"
            for column in testset.columns:
                if "target" != column[:6]:
                    if basic_column_name == column[:len(basic_column_name)] and divider_column_name != column:
                        testset[column] /= testset[divider_column_name]
                else:
                    if basic_column_name == column[7:7+len(basic_column_name)]:
                        testset[column] /= testset[divider_column_name]
            testset[divider_column_name] = 1.0

    def regression_to_classification_testset(self, testset):
        for basic_column_name in self.data.columns:
            last_observed_value_column_name = basic_column_name + ".0"
            for column in testset.columns:
                if "target" == column[:6] and basic_column_name == column[7:7+len(basic_column_name)]:
                        testset[column] = (testset[last_observed_value_column_name] < testset[column]).astype(int)

    def normalize_testset(self, testset):
        for column in testset.columns: 
            if self.data_is_normalized[column]:
                testset[column] = (testset[column] - self.data_mean[column]) / self.data_std[column]    

    def denormalize_testset(self, testset):
        for column in testset.columns: 
            if self.data_is_normalized[column]:
                testset[column] = testset[column] * self.data_std[column] + self.data_mean[column]
    
    def testset_series_to_supervised(self, testset, dropnan=True):
        n_vars = 1 if type(testset) is list else testset.shape[1]
        cols = list()
        for i in range(self.n_in, 0, -1):
            cols.append(testset.shift(i))
        for i in range(0, self.n_out):
            cols.append(testset.shift(-i))
        agg = pd.concat(cols, axis=1)
        if dropnan:
            agg.dropna(inplace=True)
        self.__rename_supervised_column_names(agg, n_vars, self.n_in, self.n_out, self.target_columns)
        return agg

In [ ]:
class Regressor:
    def __init__(self, trainDataObject, model=xgb.XGBRegressor()):
        self.dataseries = trainDataObject
        self.model = model
        
    def change_model(self, model):
        self.model = model

    def change_data(self, trainDataObject):
        self.dataseries = trainDataObject

    def walk_forward_validation(self, testset, normalize=False):
        if normalize:
            self.dataseries.normalize_testset(testset)
        testset = self.dataseries.testset_series_to_supervised(testset, dropnan=True)
        predictions = list()
        history = [x for x in self.data.data_supervised]
        for i in range(len(testset)):
            testX, testy = testset[i, :-1], testset[i, -1]
            yhat = self.forecast(history, testX)
            predictions.append(yhat)
            history.append(testset[i])
        error = sklearn.metrics.mean_absolute_error(testset[:, -1], predictions)
        return error,  testset[:, -1], predictions

    def forecast(self, train, testX):
        train = np.asarray(train)
        trainX, trainy = train[:, :-1], train[:, -1]
        self.model.fit(trainX, trainy)
        yhat = self.model.predict([testX])
        return yhat[0]

    def fit(self, n_in=50, n_out=1, dropnan=True, normalize=False, target_columns=["price"], classification=True):
        if normalize:
            self.dataseries.normalize_columns()
        if self.dataseries.data_supervised is None:
            self.dataseries.series_to_supervised(n_in=n_in, n_out=n_out, dropnan=dropnan, target_columns=target_columns)
            self.dataseries.normalize_window()
        if classification:
            self.dataseries.regression_to_classification()
        trainX = self.dataseries.data_supervised.iloc[:, :-n_out*len(target_columns)]
        trainy = self.dataseries.data_supervised.iloc[:, -n_out*len(target_columns):]
        if classification:
            trainy = trainy.to_numpy().ravel()
        self.model.fit(trainX, trainy)

    def predict(self, testset, normalize=False, classification=True):
        if normalize:
            self.dataseries.normalize_testset(testset)
        testset = self.dataseries.testset_series_to_supervised(testset, dropnan=True)
        self.dataseries.normalize_window_testset(testset)
        if classification:
            self.dataseries.regression_to_classification_testset(testset)
        predictions = self.model.predict(testset.iloc[:, :-self.dataseries.n_out*len(self.dataseries.target_columns)])
        error = sklearn.metrics.mean_absolute_error(testset.iloc[:, -self.dataseries.n_out*len(self.dataseries.target_columns):], predictions)
        return error, testset.iloc[:, -self.dataseries.n_out*len(self.dataseries.target_columns)], predictions

In [ ]:
date_split = dateutil.parser.parse("2021-11-28")
trainset = boll_df[boll_df.index <= date_split]
testset = boll_df[boll_df.index > date_split]

In [ ]:
trainObject = DataSeries(trainset)
# trainObject.normalize_columns()
trainObject.series_to_supervised(n_in=50, n_out=1, dropnan=True)
trainObject.normalize_window()
trainObject.regression_to_classification()

In [ ]:
trainObject.data_supervised

,price.-49,price.-48,price.-47,price.-46,price.-45,price.-44,price.-43,price.-42,price.-41,price.-40,price.-39,price.-38,price.-37,price.-36,price.-35,price.-34,price.-33,price.-32,price.-31,price.-30,price.-29,price.-28,price.-27,price.-26,price.-25,price.-24,price.-23,price.-22,price.-21,price.-20,price.-19,price.-18,price.-17,price.-16,price.-15,price.-14,price.-13,price.-12,price.-11,price.-10,price.-9,price.-8,price.-7,price.-6,price.-5,price.-4,price.-3,price.-2,price.-1,price.0,target.price.0
2021-11-01 10:50:00,0.998551,0.998866,0.999056,0.998531,0.998517,0.998856,0.998873,0.999161,0.998956,0.998926,0.999127,0.998926,0.999080,0.998880,0.998731,0.997843,0.997423,0.997621,0.996893,0.997223,0.998566,0.998979,0.999007,0.998707,0.998723,0.998954,0.999195,0.997731,0.998476,0.998971,0.999517,0.999517,0.999678,0.999677,0.999601,0.999601,0.999511,0.999386,0.999646,0.999839,0.999967,0.999592,0.998873,0.998464,0.998551,0.999656,0.999450,0.999264,0.999952,1.0,1
2021-11-01 10:51:00,0.998384,0.998574,0.998049,0.998035,0.998373,0.998391,0.998679,0.998474,0.998443,0.998645,0.998444,0.998598,0.998398,0.998249,0.997361,0.996942,0.997139,0.996412,0.996742,0.998084,0.998497,0.998525,0.998225,0.998241,0.998471,0.998713,0.997249,0.997994,0.998489,0.999035,0.999035,0.999195,0.999195,0.999118,0.999118,0.999028,0.998903,0.999163,0.999356,0.999484,0.999110,0.998391,0.997982,0.998069,0.999173,0.998968,0.998781,0.999469,0.999517,1.0,0
2021-11-01 10:52:00,0.998574,0.998049,0.998035,0.998374,0.998391,0.998679,0.998474,0.998444,0.998645,0.998444,0.998598,0.998398,0.998249,0.997361,0.996942,0.997139,0.996412,0.996742,0.998084,0.998497,0.998525,0.998225,0.998241,0.998472,0.998713,0.997249,0.997994,0.998489,0.999035,0.999035,0.999196,0.999195,0.999118,0.999118,0.999028,0.998903,0.999163,0.999357,0.999484,0.999110,0.998391,0.997982,0.998070,0.999173,0.998968,0.998781,0.999469,0.999517,1.000000,1.0,0
2021-11-01 10:53:00,0.998049,0.998035,0.998374,0.998391,0.998679,0.998474,0.998444,0.998645,0.998444,0.998598,0.998398,0.998249,0.997361,0.996942,0.997139,0.996412,0.996742,0.998084,0.998497,0.998525,0.998225,0.998241,0.998472,0.998713,0.997249,0.997994,0.998489,0.999035,0.999035,0.999196,0.999195,0.999118,0.999118,0.999028,0.998903,0.999163,0.999357,0.999484,0.999110,0.998391,0.997982,0.998070,0.999173,0.998968,0.998781,0.999469,0.999517,1.000000,1.000000,1.0,1
2021-11-01 10:54:00,0.997501,0.997839,0.997857,0.998145,0.997940,0.997909,0.998110,0.997910,0.998063,0.997864,0.997715,0.996827,0.996409,0.996606,0.995879,0.996208,0.997550,0.997963,0.997991,0.997691,0.997707,0.997937,0.998178,0.996716,0.997460,0.997955,0.998500,0.998500,0.998661,0.998660,0.998584,0.998584,0.998494,0.998369,0.998629,0.998822,0.998949,0.998575,0.997857,0.997448,0.997535,0.998638,0.998434,0.998247,0.998934,0.998983,0.999465,0.999465,0.999465,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-27 23:56:00,1.005484,1.005069,1.005018,1.004886,1.005082,1.005586,1.004990,1.005129,1.005354,1.005035,1.004751,1.004534,1.004294,1.003489,1.003784,1.004301,1.003319,1.002525,1.002782,1.001877,1.001876,1.002752,1.002436,1.002435,1.002690,1.001625,1.001674,1.002291,1.002330,1.002280,1.001220,1.001553,1.001666,1.001598,1.001014,1.000970,1.000818,1.000850,1.000712,1.000314,0.999488,0.998685,0.998627,0.997756,0.998050,0.998447,0.999442,0.999834,0.999652,1.0,1
2021-11-27 23:57:00,1.004350,1.004299,1.004167,1.004362,1.004866,1.004271,1.004410,1.004635,1.004316,1.004032,1.003815,1.003575,1.002771,1.003065,1.003582,1.002601,1.001808,1.002065,1.001160,1.001159,1.002034,1.001718,1.001718,1.001973,1.000908,1.000958,1.001573,1.001613,1.001563,1.000504,1.000837,1.000949,1.000881,1.000298,1.000254,1.000102,1.000134,0.999996,0.999599,0.998773,0.997970,0.997912,0.997042,0.997336,0.997733,0.998727,0.999118,0.998937,0.999284,1.0,1
2021-11-27 23:58:0

In [ ]:
trainObject.data

,price
2021-11-01 10:00:00,62030.00
2021-11-01 10:01:00,62049.54
2021-11-01 10:02:00,62061.35
2021-11-01 10:03:00,62028.72
2021-11-01 10:04:00,62027.86
...,...
2021-11-27 23:56:00,54572.61
2021-11-27 23:57:00,54615.60
2021-11-27 23:58:00,54819.29
2021-11-27 23:59:00,54834.27


In [ ]:
reg = Regressor(trainObject, model=xgb.XGBClassifier())
reg.fit()